In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential
from functions import mean_gen, varx_gen, vary_gen, spherical_data, test_on_integers, learn_parameters

2022-08-10 15:05:36.714199: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-10 15:05:36.714233: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Random Seed
rand_n = 12345

# Single Point on Sphere
n = 40000
angles = np.array([1.3, 1.7])
thetas, phis = angles[0] * np.ones(n), angles[1] * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)

In [3]:
model_interpolate = tf.keras.models.load_model('3dmodels/discrete_model_mth50_mph50')

for l in model_interpolate.layers:
    l.trainable=False
    
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Building model_angles which is used to train (theta, phi)
inputs_hold = tf.keras.Input(shape=(1,))
simple_linear = Dense(2, use_bias = False, kernel_initializer='ones')(inputs_hold)
model_angles = Model(inputs = inputs_hold, outputs = simple_linear)

# Building model_parmafinder, inputs, which takes the (x, y) and finds the best (theta, phi)
raw_inputs = tf.keras.Input(shape=(2,))
inputs = tf.keras.layers.concatenate([raw_inputs, model_angles(tf.ones_like(raw_inputs)[:,0:1])])
output = model_interpolate(inputs)

model_paramfinder = Model(inputs = raw_inputs, outputs = output)
model_paramfinder.compile(loss=loss_fn, optimizer='Adam')
model_paramfinder.summary()

2022-08-10 15:05:42.425196: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-10 15:05:42.425228: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-10 15:05:42.425249: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-matan): /proc/driver/nvidia/version does not exist
2022-08-10 15:05:42.425586: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf.ones_like (TFOpLambda)       (None, 2)            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 1)            0           tf.ones_like[0][0]               
__________________________________________________________________________________________________
model (Functional)              (None, 2)            2           tf.__operators__.getitem[0][0]   
____________________________________________________________________________________________

In [4]:
test_on_integers(x_train, y_train)

discrete_model_th5_ph3 has loss 0.6751845678870159
discrete_model_th3_ph3 has loss 0.6679355034291584
discrete_model_th2_ph2 has loss 0.5877782896867955
discrete_model_th1_ph1 has loss 0.6266451570210229
discrete_model_th0_ph0 has loss 0.6942435518819838
discrete_model_mth40_mph40 has loss 0.57290454970477


In [5]:
epochs, iterations, batch_size = 1, 50, 100

weights = learn_parameters(model_paramfinder, x_train, y_train, iterations, epochs, batch_size)
angle_errors = (angles - weights) / angles
average_error = np.sum(np.abs(angle_errors)) / 2
print('Average error %:', average_error * 100)

2022-08-10 15:05:56.209897: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


640/640 [==============================] - 2s 2ms/step - loss: 0.5861
0 Fitted result:  tf.Tensor([1.2763753 1.3937037], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.5758
1 Fitted result:  tf.Tensor([1.300679  1.5269145], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.5748
2 Fitted result:  tf.Tensor([1.3203766 1.5680984], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.5751
3 Fitted result:  tf.Tensor([1.3380346 1.589419 ], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.5748
4 Fitted result:  tf.Tensor([1.3328549 1.6004454], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.5749
5 Fitted result:  tf.Tensor([1.3268574 1.5907892], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.5742
6 Fitted result:  tf.Tensor([1.3299806 1.5

In [6]:
xr = model_interpolate(x_train).numpy().transpose()[0]
y = model_interpolate(x_train)
loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
print(loss)

0.5721967718613304


In [ ]:
sort(xr)

In [7]:
x_train_mod = np.copy(x_train)
x_train_mod[:, 2] = x_train[:, 2]
x_train_mod[:, 3] = x_train[:, 3] - .1
xr_mod = model_interpolate(x_train_mod).numpy().transpose()[0]
y_mod = model_interpolate(x_train_mod)
loss_mod = np.sum(-y_train * np.log(xr_mod) - (1-y_train) * np.log(1-xr_mod)) / xr_mod.shape[0]
print(loss_mod)

0.5717564871746726


# x_train